In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import re
import json
import seaborn as sns
import math

In [16]:
raw_communities = pd.read_csv("../Data/communities.csv")
raw_egm = pd.read_csv("../Data/EGM.csv")
raw_housingPrice = pd.read_csv("../Data/Houses-by-suburb.csv")
raw_all_offences = pd.read_excel("../Data/LGA_Offences.xlsx", sheet_name=None)
raw_offences = raw_all_offences['Table 01']

communities = raw_communities.copy()
egm = raw_egm.copy()
housingPrice = raw_housingPrice.copy()
offences = raw_offences.copy()

In [17]:
# Clean the LGA of the 3 tables (Housing price is special)
def clean_lga_tables(communities, egm, offences):
    def clean_lga(lga: str):
        lga = str(lga)
        lga = lga.strip().lower()
        lga = lga.replace('-', '')
        replaces = {
            'moreland': 'merribek'
        }
        lga = ''.join([replaces.get(word, word) for word in lga.split() if word not in ['of', 'city', 'rural', 'shire', 'borough']])
        # Remove the bracket from communities
        lga = lga.split('(')[0]
        return lga
    
    communities["LGA"] = communities["LGA"].apply(clean_lga)
    egm["LGA Name"] = egm["LGA Name"].apply(clean_lga)
    offences["Local Government Area"] = offences["Local Government Area"].apply(clean_lga)

In [18]:
# # Try to scrape the government website
# # Get the list of 79 LGA (local councils)
# def getLGAFromCouncilLink(url: str):
#     rePattern = r'/know-your-council-(.*)'
#     councilName = re.match(rePattern, url).group(1)
#     councilName = councilName.strip().lower()
#     words = councilName.split('-')
#     lga = ''.join([word for word in words if word not in ['of', 'city', 'rural', 'shire', 'borough', 'council']])
#     return lga

# prefixUrl = "https://www.vic.gov.au"
# page = requests.get("https://www.vic.gov.au/know-your-council")
# soup = BeautifulSoup(page.text, 'html.parser')

# councilList = soup.find("ul", {"class": 'vic-kyc__councils-list rpl-grid'})
# allLinksToCouncil = councilList.find_all("a")
# lgaLocality = {}

# for anchorTag in allLinksToCouncil:
#     link = anchorTag["href"]
#     lga = getLGAFromCouncilLink(link)

#     page = requests.get(prefixUrl + link)
#     localitySoup = BeautifulSoup(page.text, 'html.parser')
#     localityHeader = localitySoup.find(id="localities")
#     localityParagraph = localityHeader.find_next("p")
#     localityText = localityParagraph.text.strip(" .").replace(" and ", ", ")
#     locailityList = localityText.split(", ")

#     lgaLocality[lga] = locailityList

# # with open("lgaLocality.json", "w") as outFile:
# #     json.dump(lgaLocality, outFile, indent=4)

# lgaLocality


In [19]:
# # Try using the scraped data to map locality to LGA
# # There are missing localities, so mainly use this to find the missing localities
# with open("../lgaLocality.json", "r") as file:
#     lgaLocality = json.load(file)

# def clean_communities(community: str):
#     community = community.strip().lower()
#     # Strip off any brackets at the end
#     community = community.split("(")[0]
#     # Rip off directions
#     words = list(community.split())
#     for i in range(len(words)):
#         if words[i] in ("north", "south", "east", "west", "central"):
#             words[i] = ""
#     return ''.join(words)


# housingPrice["Locality"] = housingPrice["Locality"].apply(clean_communities)

# mapDictionary = {}
# cannotMap = []

# for housingPriceCommunity in housingPrice["Locality"]:
#     mapped = False
#     for lga in lgaLocality:
#         for community in lgaLocality[lga]:
#             community = clean_communities(community)
#             if housingPriceCommunity in community:
#                 mapDictionary[housingPriceCommunity] = (community, lga)
#                 mapped = True
#                 break
#         if mapped:
#             break
#     if not mapped:
#         cannotMap.append(housingPriceCommunity)

# print(mapDictionary)
# print(cannotMap)
# print(mapDictionary['osborne'])


In [20]:
# # Compute the full list to convert community to LGA
# def clean_communities(community: str):
#     community = community.strip().lower()
#     # Strip off any brackets at the end
#     community = community.split("(")[0]
#     # Rip off directions
#     words = list(community.split())
#     for i in range(len(words)):
#         if words[i] in ("north", "south", "east", "west", "central"):
#             words[i] = ""
#     return ''.join(words)

# with open("../lgaLocality.json", "r") as file:
#     lgaLocality = json.load(file)

# handMapLocalityToLGA = {'aintree': 'melton', 
#                       'balcombe': 'wodonga', 
#                       'benalla': 'benalla', 
#                       'bonniebrook': 'melton', 
#                       'botanicridge': 'casey', 
#                       'capelsound': 'morningtonpeninsula', 
#                       'cobblebank': 'melton', 
#                       'coonanshill': 'merribek', 
#                       'danyo': 'mildura', 
#                       'deanside': 'melton', 
#                       'deepdene': 'boroondara', 
#                       'eastwood': 'eastgippsland', 
#                       'fiveways': 'latrobe', 
#                       'fraserrise': 'melton', 
#                       'glenwaverley': 'monash', 
#                       'harkness': 'melton', 
#                       'jolimont': 'greatershepparton', 
#                       'laralake': 'greatergeelong', 
#                       'lucas': 'ballarat', 
#                       'manorlakes': 'wyndham', 
#                       'mansfield': 'mansfield', 
#                       'mckenziehill': 'mountalexander', 
#                       'merindapark': 'casey', 
#                       'pattersongardens': 'brimbank', 
#                       'sanctuarylakes': 'wyndham', 
#                       'sthelena': 'banyule', 
#                       'strathtulloh': 'melton', 
#                       'thornhillpark': 'melton', 
#                       'weirviews': 'melton', 
#                       'westall': 'greaterdandenong', 
#                       'westgarth': 'nillumbik', 
#                       'wimbledonheights': 'basscoast', 
#                       'wintervalley': 'ballarat', 
#                       'woolamaiwaters': 'basscoast'}
# localityToLGA = {}
# for community in mapDictionary:
#     localityToLGA[community] = mapDictionary[community][1]

# for community in handMapLocalityToLGA:
#     localityToLGA[community] = handMapLocalityToLGA[community]
# with open("../localityToLGA.json", "w") as outFile:
#     json.dump(localityToLGA, outFile, indent=4)

# print(len(localityToLGA))

In [21]:
def clean_lga_housing(housingPrice):
    with open("../localityToLGA.json", "r") as file:
        localityToLGAMap = json.load(file)


    def clean_communities(community: str):
        community = community.strip().lower()
        # Strip off any brackets at the end
        community = community.split("(")[0]
        # Rip off directions
        words = list(community.split())
        for i in range(len(words)):
            if words[i] in ("north", "south", "east", "west", "central"):
                words[i] = ""
        return ''.join(words)

    def localityToLGA(locality: str, localityToLGAMap: dict):
        return localityToLGAMap[clean_communities(locality)]


    housingPrice["LGA"] = housingPrice["Locality"].apply(lambda x: localityToLGA(x, localityToLGAMap))

# clean_lga_housing(housingPrice)
# housingPrice["LGA"]

In [22]:
# Preprocess
def clean_lga_all(communities, egm, housingPrice, offences):
    clean_lga_tables(communities=communities, egm=egm, offences=offences)
    clean_lga_housing(housingPrice)

clean_lga_all(communities, egm, housingPrice, offences)

In [23]:
# Clean egm (mainly check for type)
for column in egm.columns:
    print(column, ":", egm[column].dtype)

LGA Name : object
Region : object
2011 : float64
2012 : float64
2013 : float64
2014 : float64
2015 : float64
2016 : float64
2017 : float64
2018 : float64
2019 : float64
2020 : float64


In [24]:
# Clean offenses
# Type check
for column in offences.columns:
    print(column, ":", offences[column].dtype)

# Drop NA for the one without rate (total, unincorporated, etc.), since they are not relevant
offences = offences.dropna()

# offences.isna().sum()

# Remove comma to convert it to float
offences["Offence Count"] = offences["Offence Count"].astype("string")
offences["Rate per 100,000 population"] = offences["Rate per 100,000 population"].astype("string")

offences["Offence Count"] = offences["Offence Count"].apply(lambda x: x.replace(",", ""))
offences["Rate per 100,000 population"] = offences["Rate per 100,000 population"].apply(lambda x: x.replace(",", ""))

offences["Offence Count"] = offences["Offence Count"].astype("float")
offences["Rate per 100,000 population"] = offences["Rate per 100,000 population"].astype("float")

for column in offences.columns:
    print(column, ":", offences[column].dtype)

Year : int64
Year ending : object
Police Region : object
Local Government Area : object
Offence Count : int64
Rate per 100,000 population : float64
Year : int64
Year ending : object
Police Region : object
Local Government Area : object
Offence Count : float64
Rate per 100,000 population : float64


In [25]:
# Clean communities
# Pick the features to process
# LGA (alp): Id, grouped by LGA
# Location (num): Derive x and y distance from Melbourne, and use average
# Area (km^2) (num) 

# Remove all % columns, or convert to raw if not available
# Remove 2007 columns (if don't want to extrapolate)
# Clean the <5 in all columns
# min, max, avg: Use all for now until correlation analysis


In [26]:
# Filtered columns
columns_keep = [
  # 'Community Name',
  # 'Region',
  # 'Map reference',
  # 'Grid reference',
  'Location',
  'Population Density',
  'Travel time to GPO (minutes)',
  # 'Distance to GPO (km)',
  'LGA',
  # 'Primary Care Partnership',
  # 'Medicare Local',
  'Area (km^2)',
  'ARIA+ (min)',
  'ARIA+ (max)',
  'ARIA+ (avg)',
  'ABS remoteness category',
  # 'DHS Area',
  'Commercial (km^2)',
  # 'Commercial (%)',
  'Industrial (km^2)',
  # 'Industrial (%)',
  'Residential (km^2)',
  # 'Residential (%)',
  'Rural (km^2)',
  # 'Rural (%)',
  'Other (km^2)',
  # 'Other (%)',
  # '2012 ERP age 0-4, persons',
  # '2012 ERP age 0-4, %',
  # '2012 ERP age 5-9, persons',
  # '2012 ERP age 5-9, %',
  # '2012 ERP age 10-14, persons',
  # '2012 ERP age 10-14, %',
  # '2012 ERP age 15-19, persons',
  # '2012 ERP age 15-19, %',
  # '2012 ERP age 20-24, persons',
  # '2012 ERP age 20-24, %',
  # '2012 ERP age 25-44, persons',
  # '2012 ERP age 25-44, %',
  # '2012 ERP age 45-64, persons',
  # '2012 ERP age 45-64, %',
  # '2012 ERP age 65-69, persons',
  # '2012 ERP age 65-69, %',
  # '2012 ERP age 70-74, persons',
  # '2012 ERP age 70-74, %',
  # '2012 ERP age 75-79, persons',
  # '2012 ERP age 75-79, %',
  # '2012 ERP age 80-84, persons',
  # '2012 ERP age 80-84, %',
  # '2012 ERP age 85+, persons',
  # '2012 ERP age 85+, %',
  '2012 ERP, total',
  # '2007 ERP age 0-4, persons',
  # '2007 ERP age 0-4, %',
  # '2007 ERP age 5-9, persons',
  # '2007 ERP age 5-9, %',
  # '2007 ERP age 10-14, persons',
  # '2007 ERP age 10-14, %',
  # '2007 ERP age 15-19, persons',
  # '2007 ERP age 15-19, %',
  # '2007 ERP age 20-24, persons',
  # '2007 ERP age 20-24, %',
  # '2007 ERP age 25-44, persons',
  # '2007 ERP age 25-44, %',
  # '2007 ERP age 45-64, persons',
  # '2007 ERP age 45-64, %',
  # '2007 ERP age 65-69, persons',
  # '2007 ERP age 65-69, %',
  # '2007 ERP age 70-74, persons',
  # '2007 ERP age 70-74, %',
  # '2007 ERP age 75-79, persons',
  # '2007 ERP age 75-79, %',
  # '2007 ERP age 80-84, persons',
  # '2007 ERP age 80-84, %',
  # '2007 ERP age 85+, persons',
  # '2007 ERP age 85+, %',
  '2007 ERP, total',
  # '% change, 2007-2012, age 0-4',
  # '% change, 2007-2012, age 5-9',
  # '% change, 2007-2012, age 10-14',
  # '% change, 2007-2012, age 15-19',
  # '% change, 2007-2012, age 20-24',
  # '% change, 2007-2012, age 25-44',
  # '% change, 2007-2012, age 45-64',
  # '% change, 2007-2012, age 65-69',
  # '% change, 2007-2012, age 70-74',
  # '% change, 2007-2012, age 75-79',
  # '% change, 2007-2012, age 80-84',
  # '% change, 2007-2012, age 85+',
  # '% change, 2007-2012, total',
  'Public Hospitals',
  'Private Hospitals',
  'Community Health Centres',
  'Bush Nursing Centres',
  'Allied Health',
  'Alternative Health',
  'Child Protection and Family',
  'Dental',
  'Disability',
  'General Practice',
  'Homelessness',
  'Mental Health',
  'Pharmacies',
  'Aged Care (High Care)',
  'Aged Care (Low Care)',
  'Aged Care (SRS)',
  'Kinder and/or Childcare',
  'Primary Schools',
  'Secondary Schools',
  'P12 Schools',
  'Other Schools',
  'Centrelink Offices',
  'Medicare Offices',
  'Medicare Access Points',
  'Number of Households',
  # 'Average persons per household',
  'Occupied private dwellings',
  # 'Occupied private dwellings, %',
  'Population in non-private dwellings',
  'Public Housing Dwellings',
  # '% dwellings which are public housing',
  'Dwellings with no motor vehicle',
  # 'Dwellings with no motor vehicle, %',
  'Dwellings with no internet',
  # 'Dwellings with no internet, %',
  'Equivalent household income <$600/week',
  # 'Equivalent household income <$600/week, %',
  'Personal income <$400/week, persons',
  # 'Personal income <$400/week, %',
  'Number of families',
  'Female-headed lone parent families',
  # 'Female-headed lone parent families, %',
  'Male-headed lone parent families',
  # 'Male-headed lone parent families, %',
  '% residing near PT',
  'IRSD (min)',
  'IRSD (max)',
  'IRSD (avg)',
  'Primary school students',
  'Secondary school students',
  'TAFE students',
  'University students',
  'Holds degree or higher, persons',
  # 'Holds degree or higher, %',
  'Did not complete year 12, persons',
  # 'Did not complete year 12, %',
  'Unemployed, persons',
  # 'Unemployed, %',
  'Volunteers, persons',
  # 'Volunteers, %',
  'Requires assistance with core activities, persons',
  # 'Requires assistance with core activities, %',
  'Aged 75+ and lives alone, persons',
  # 'Aged 75+ and lives alone, %',
  'Unpaid carer to person with disability, persons',
  # 'Unpaid carer to person with disability, %',
  'Unpaid carer of children, persons',
  # 'Unpaid carer of children, %',
  # 'Top industry',
  # 'Top industry, %',
  # '2nd top industry - persons',
  # '2nd top industry, %',
  # '3rd top industry - persons',
  # '3rd top industry, %',
  # 'Top occupation',
  # 'Top occupation, %',
  # '2nd top occupation - persons',
  # '2nd top occupation, %',
  # '3rd top occupation - persons',
  # '3rd top occupation, %',
  'Aboriginal or Torres Strait Islander, persons',
  # 'Aboriginal or Torres Strait Islander, %',
  'Born overseas, persons',
  # 'Born overseas, %',
  'Born in non-English speaking country, persons',
  # 'Born in non-English speaking country, %',
  'Speaks LOTE at home, persons',
  # 'Speaks LOTE at home, %',
  'Poor English proficiency, persons',
  # 'Poor English proficiency, %',
  # 'Top country of birth',
  # 'Top country of birth, persons',
  # 'Top country of birth, %',
  # '2nd top country of birth',
  # '2nd top country of birth, persons',
  # '2nd top country of birth, %',
  # '3rd top country of birth',
  # '3rd top country of birth, persons',
  # '3rd top country of birth, %',
  # '4th top country of birth',
  # '4th top country of birth, persons',
  # '4th top country of birth, %',
  # '5th top country of birth',
  # '5th top country of birth, persons',
  # '5th top country of birth, %',
  # 'Top language spoken',
  # 'Top language spoken, persons',
  # 'Top language spoken, %',
  # '2nd top language spoken',
  # '2nd top language spoken, persons',
  # '2nd top language spoken, %',
  # '3rd top language spoken',
  # '3rd top language spoken, persons',
  # '3rd top language spoken, %',
  # '4th top language spoken',
  # '4th top language spoken, persons',
  # '4th top language spoken, %',
  # '5th top language spoken',
  # '5th top language spoken, persons',
  # '5th top language spoken, %',
  # 'Public hospital separations, 2012-13',
  # 'Nearest Public Hospital',
  'Travel time to nearest public hospital',
  # 'Distance to nearest public hospital',
  # 'Obstetric type separations, 2012-13',
  # 'Nearest public hospital with maternity services',
  'Time to nearest public hospital with maternity services',
  # 'Distance to nearest public hospital with maternity services',
  'Presentations to emergency departments, 2012-13',
  # 'Nearest public hospital with emergency department',
  'Travel time to nearest public hospital with emergency department',
  # 'Distance to nearest public hospital with emergency department',
  'Presentations to emergency departments due to injury',
  # 'Presentations to emergency departments due to injury, %',
  'Category 4 & 5 emergency department presentations',
  # 'Category 4 & 5 emergency department presentations, %'
]

columns_derive = [
  # 'Community Name',
  # 'Region',
  # 'Map reference',
  # 'Grid reference',
  # 'Location',
  # 'Population Density',
  # 'Travel time to GPO (minutes)',
  # 'Distance to GPO (km)',
  # 'LGA',
  # 'Primary Care Partnership',
  # 'Medicare Local',
  # 'Area (km^2)',
  # 'ARIA+ (min)',
  # 'ARIA+ (max)',
  # 'ARIA+ (avg)',
  # 'ABS remoteness category',
  # 'DHS Area',
  # 'Commercial (km^2)',
  'Commercial (%)',
  # 'Industrial (km^2)',
  'Industrial (%)',
  # 'Residential (km^2)',
  'Residential (%)',
  # 'Rural (km^2)',
  'Rural (%)',
  # 'Other (km^2)',
  'Other (%)',
  # '2012 ERP age 0-4, persons',
  # '2012 ERP age 0-4, %',
  # '2012 ERP age 5-9, persons',
  # '2012 ERP age 5-9, %',
  # '2012 ERP age 10-14, persons',
  # '2012 ERP age 10-14, %',
  # '2012 ERP age 15-19, persons',
  # '2012 ERP age 15-19, %',
  # '2012 ERP age 20-24, persons',
  # '2012 ERP age 20-24, %',
  # '2012 ERP age 25-44, persons',
  # '2012 ERP age 25-44, %',
  # '2012 ERP age 45-64, persons',
  # '2012 ERP age 45-64, %',
  # '2012 ERP age 65-69, persons',
  # '2012 ERP age 65-69, %',
  # '2012 ERP age 70-74, persons',
  # '2012 ERP age 70-74, %',
  # '2012 ERP age 75-79, persons',
  # '2012 ERP age 75-79, %',
  # '2012 ERP age 80-84, persons',
  # '2012 ERP age 80-84, %',
  # '2012 ERP age 85+, persons',
  # '2012 ERP age 85+, %',
  # '2012 ERP, total',
  # '2007 ERP age 0-4, persons',
  # '2007 ERP age 0-4, %',
  # '2007 ERP age 5-9, persons',
  # '2007 ERP age 5-9, %',
  # '2007 ERP age 10-14, persons',
  # '2007 ERP age 10-14, %',
  # '2007 ERP age 15-19, persons',
  # '2007 ERP age 15-19, %',
  # '2007 ERP age 20-24, persons',
  # '2007 ERP age 20-24, %',
  # '2007 ERP age 25-44, persons',
  # '2007 ERP age 25-44, %',
  # '2007 ERP age 45-64, persons',
  # '2007 ERP age 45-64, %',
  # '2007 ERP age 65-69, persons',
  # '2007 ERP age 65-69, %',
  # '2007 ERP age 70-74, persons',
  # '2007 ERP age 70-74, %',
  # '2007 ERP age 75-79, persons',
  # '2007 ERP age 75-79, %',
  # '2007 ERP age 80-84, persons',
  # '2007 ERP age 80-84, %',
  # '2007 ERP age 85+, persons',
  # '2007 ERP age 85+, %',
  # '2007 ERP, total',
  # '% change, 2007-2012, age 0-4',
  # '% change, 2007-2012, age 5-9',
  # '% change, 2007-2012, age 10-14',
  # '% change, 2007-2012, age 15-19',
  # '% change, 2007-2012, age 20-24',
  # '% change, 2007-2012, age 25-44',
  # '% change, 2007-2012, age 45-64',
  # '% change, 2007-2012, age 65-69',
  # '% change, 2007-2012, age 70-74',
  # '% change, 2007-2012, age 75-79',
  # '% change, 2007-2012, age 80-84',
  # '% change, 2007-2012, age 85+',
  # '% change, 2007-2012, total',
  # 'Public Hospitals',
  # 'Private Hospitals',
  # 'Community Health Centres',
  # 'Bush Nursing Centres',
  # 'Allied Health',
  # 'Alternative Health',
  # 'Child Protection and Family',
  # 'Dental',
  # 'Disability',
  # 'General Practice',
  # 'Homelessness',
  # 'Mental Health',
  # 'Pharmacies',
  # 'Aged Care (High Care)',
  # 'Aged Care (Low Care)',
  # 'Aged Care (SRS)',
  # 'Kinder and/or Childcare',
  # 'Primary Schools',
  # 'Secondary Schools',
  # 'P12 Schools',
  # 'Other Schools',
  # 'Centrelink Offices',
  # 'Medicare Offices',
  # 'Medicare Access Points',
  # 'Number of Households',
  'Average persons per household',
  # 'Occupied private dwellings',
  'Occupied private dwellings, %',
  # 'Population in non-private dwellings',
  # 'Public Housing Dwellings',
  '% dwellings which are public housing',
  # 'Dwellings with no motor vehicle',
  'Dwellings with no motor vehicle, %',
  # 'Dwellings with no internet',
  'Dwellings with no internet, %',
  # 'Equivalent household income <$600/week',
  'Equivalent household income <$600/week, %',
  # 'Personal income <$400/week, persons',
  'Personal income <$400/week, %',
  # 'Number of families',
  # 'Female-headed lone parent families',
  'Female-headed lone parent families, %',
  # 'Male-headed lone parent families',
  'Male-headed lone parent families, %',
  # '% residing near PT',
  # 'IRSD (min)',
  # 'IRSD (max)',
  # 'IRSD (avg)',
  # 'Primary school students',
  # 'Secondary school students',
  # 'TAFE students',
  # 'University students',
  # 'Holds degree or higher, persons',
  'Holds degree or higher, %',
  # 'Did not complete year 12, persons',
  'Did not complete year 12, %',
  # 'Unemployed, persons',
  'Unemployed, %',
  # 'Volunteers, persons',
  'Volunteers, %',
  # 'Requires assistance with core activities, persons',
  'Requires assistance with core activities, %',
  # 'Aged 75+ and lives alone, persons',
  'Aged 75+ and lives alone, %',
  # 'Unpaid carer to person with disability, persons',
  'Unpaid carer to person with disability, %',
  # 'Unpaid carer of children, persons',
  'Unpaid carer of children, %',
  # 'Top industry',
  # 'Top industry, %',
  # '2nd top industry - persons',
  # '2nd top industry, %',
  # '3rd top industry - persons',
  # '3rd top industry, %',
  # 'Top occupation',
  # 'Top occupation, %',
  # '2nd top occupation - persons',
  # '2nd top occupation, %',
  # '3rd top occupation - persons',
  # '3rd top occupation, %',
  # 'Aboriginal or Torres Strait Islander, persons',
  'Aboriginal or Torres Strait Islander, %',
  # 'Born overseas, persons',
  'Born overseas, %',
  # 'Born in non-English speaking country, persons',
  'Born in non-English speaking country, %',
  # 'Speaks LOTE at home, persons',
  # 'Speaks LOTE at home, %',
  # 'Poor English proficiency, persons',
  # 'Poor English proficiency, %',
  # 'Top country of birth',
  # 'Top country of birth, persons',
  # 'Top country of birth, %',
  # '2nd top country of birth',
  # '2nd top country of birth, persons',
  # '2nd top country of birth, %',
  # '3rd top country of birth',
  # '3rd top country of birth, persons',
  # '3rd top country of birth, %',
  # '4th top country of birth',
  # '4th top country of birth, persons',
  # '4th top country of birth, %',
  # '5th top country of birth',
  # '5th top country of birth, persons',
  # '5th top country of birth, %',
  # 'Top language spoken',
  # 'Top language spoken, persons',
  # 'Top language spoken, %',
  # '2nd top language spoken',
  # '2nd top language spoken, persons',
  # '2nd top language spoken, %',
  # '3rd top language spoken',
  # '3rd top language spoken, persons',
  # '3rd top language spoken, %',
  # '4th top language spoken',
  # '4th top language spoken, persons',
  # '4th top language spoken, %',
  # '5th top language spoken',
  # '5th top language spoken, persons',
  # '5th top language spoken, %',
  # 'Public hospital separations, 2012-13',
  # 'Nearest Public Hospital',
  # 'Travel time to nearest public hospital',
  # 'Distance to nearest public hospital',
  # 'Obstetric type separations, 2012-13',
  # 'Nearest public hospital with maternity services',
  # 'Time to nearest public hospital with maternity services',
  # 'Distance to nearest public hospital with maternity services',
  # 'Presentations to emergency departments, 2012-13',
  # 'Nearest public hospital with emergency department',
  # 'Travel time to nearest public hospital with emergency department',
  # 'Distance to nearest public hospital with emergency department',
  # 'Presentations to emergency departments due to injury',
  'Presentations to emergency departments due to injury, %',
  # 'Category 4 & 5 emergency department presentations',
  'Category 4 & 5 emergency department presentations, %'
]

In [27]:
# Drop the unincorporatedvic - Not a proper LGA
communities = communities[communities["LGA"] != "unincorporatedvic"].copy()

In [28]:
# Get the number of dwellings out of the occupied private dwellings 
# to be able to combine among groups into 1 %
columns_keep.append("Number of Dwellings")
communities["Number of Dwellings"] = communities["Occupied private dwellings"] / communities["Occupied private dwellings, %"] * 100

In [29]:
# Drop all unneccessary and derived columns
allColumns = communities.columns
dropColumns = []
for column in allColumns:
    if column not in columns_keep:
        dropColumns.append(column)

communities = communities.drop(columns=dropColumns)

In [30]:
# Get the population and drop the population density
columns_keep.append("Population")
columns_keep.remove("Population Density")
communities["Population"] = communities["Population Density"] * communities["Area (km^2)"]


In [31]:
# Transform location to X and Y coordinates
# Also update the location column
columns_keep.remove("Location")
columns_keep.append("Location_x")
columns_keep.append("Location_y")

DIRECTION_ANGLES = {
    "E": 0,
    "ENE": 22.5,
    "NE": 45,
    "NNE": 67.5,
    "N": 90,
    "NNW": 112.5,
    "NW": 135,
    "WNW": 157.5,
    "W": 180,
    "WSW": 202.5,
    "SW": 225,
    "SSW": 247.5,
    "S": 270,
    "SSE": 292.5,
    "SE": 315,
    "ESE": 337.5
}

def extractLocation(locationStr):
    locationRegex = r"(\d+\.?\d*)km (.+) of Melbourne"
    matchObj = re.match(locationRegex, locationStr)
    if matchObj == None:
        print(locationStr)
    distance = float(matchObj.group(1))
    direction = matchObj.group(2)
    return (distance, direction)

def getX(locationTuple):
    distance, direction = locationTuple
    return distance * math.cos(math.radians(DIRECTION_ANGLES[direction]))

def getY(locationTuple):
    distance, direction = locationTuple
    return distance * math.sin(math.radians(DIRECTION_ANGLES[direction]))

communities["Location_x"] = communities["Location"].apply(lambda x: getX(extractLocation(x)))
communities["Location_y"] = communities["Location"].apply(lambda x: getY(extractLocation(x)))
communities = communities.drop(columns=["Location"])

In [32]:
# Ordinal encode the ABS remoteness category
# Note: the ABS Remoteness is the expert binning of average ARIA
# Rather than using mode, apply the same category binnning on group average
# Drop the individual one
communities = communities.drop(columns=["ABS remoteness category"])
columns_keep.remove("ABS remoteness category")


In [33]:
# Impute the 2007 ERP total since some of the rows are wiped out
# If is NaN, assume then impute with the mean of the same lga
for idx, row in communities.iterrows():
    if pd.isna(row["2007 ERP, total"]):
        communities.loc[idx, "2007 ERP, total"] = communities.groupby("LGA")["2007 ERP, total"].mean().loc[row["LGA"]]

# If the entire group is NaN, impute the group with the 2012 ERP, total
for idx, row in communities.iterrows():
    if pd.isna(row["2007 ERP, total"]):
        communities.loc[idx, "2007 ERP, total"] = row["2012 ERP, total"]
communities["2007 ERP, total"].isna().sum()


0

In [34]:
# Make sure all kept columns, except ABS remoteness category is float
# Filtered out the "<5" values
def filterLower5(x: str):
    if x == "<5":
        return "5"
    return x

for column in columns_keep:
    if column == "ABS remoteness category" or column == "LGA":
        continue

    communities[column] = communities[column].apply(filterLower5)
    communities[column] = communities[column].astype("float")

In [35]:
# Convert all % back to ratio to avoid it messing with the normalized step
for column in columns_keep:
    if "%" in column:
        communities[column] /= 100

In [36]:
# Group by LGA
aggregate = {}

# Set aggregator method default to sum
for column in columns_keep:
    aggregate[column] = "sum"
aggregate.pop("LGA")



# Use mean for some columns
aggregate["Travel time to GPO (minutes)"] = "mean"
aggregate["Travel time to nearest public hospital"] = "mean"
aggregate["Time to nearest public hospital with maternity services"] = "mean"
aggregate["Travel time to nearest public hospital with emergency department"] = "mean"
aggregate["Location_x"] = "mean"
aggregate["Location_y"] = "mean"


# Multiplied by population/area/etc. for the weighted average
weightedPopulation = ["ARIA+ (avg)", "ARIA+ (max)", "ARIA+ (min)",
                      "IRSD (avg)", "IRSD (max)", "IRSD (min)"]
for col in weightedPopulation:
    communities[col] *= communities["Population"]


communities["% residing near PT"] *= communities["Number of families"]
# Convert percent to ratio to match the other ration
communities["% residing near PT"] /= 100

# communities[weightedPopulation]

# List of columns to be normalized by population, area, etc. (basically convert it into % of population, etc.)
# Normalized by presentation first, since the total presenations will be normalized to population
presentationNormCols = [
  'Presentations to emergency departments due to injury',
  # 'Presentations to emergency departments due to injury, %',
  'Category 4 & 5 emergency department presentations',
  # 'Category 4 & 5 emergency department presentations, %'
]

areaNormCols = [
  'Commercial (km^2)',
  # 'Commercial (%)',
  'Industrial (km^2)',
  # 'Industrial (%)',
  'Residential (km^2)',
  # 'Residential (%)',
  'Rural (km^2)',
  # 'Rural (%)',
  'Other (km^2)'
]

dwellingNormCols = [
  'Occupied private dwellings',
  # 'Occupied private dwellings, %',
  'Population in non-private dwellings',
  'Public Housing Dwellings',
  # '% dwellings which are public housing',
  'Dwellings with no motor vehicle',
  # 'Dwellings with no motor vehicle, %',
  'Dwellings with no internet'
]

householdNormCols = [
  'Equivalent household income <$600/week',
  # 'Equivalent household income <$600/week, %',
  'Personal income <$400/week, persons'
]

familyNormCols = [
  'Female-headed lone parent families',
  # 'Female-headed lone parent families, %',
  'Male-headed lone parent families',
  # 'Male-headed lone parent families, %',
  '% residing near PT'
]

populationNormCols = [
  'ARIA+ (min)',
  'ARIA+ (max)',
  'ARIA+ (avg)',
  "IRSD (avg)", 
  "IRSD (max)", 
  "IRSD (min)",
  'Public Hospitals',
  'Private Hospitals',
  'Community Health Centres',
  'Bush Nursing Centres',
  'Allied Health',
  'Alternative Health',
  'Child Protection and Family',
  'Dental',
  'Disability',
  'General Practice',
  'Homelessness',
  'Mental Health',
  'Pharmacies',
  'Aged Care (High Care)',
  'Aged Care (Low Care)',
  'Aged Care (SRS)',
  'Kinder and/or Childcare',
  'Primary Schools',
  'Secondary Schools',
  'P12 Schools',
  'Other Schools',
  'Centrelink Offices',
  'Medicare Offices',
  'Medicare Access Points',
  'Primary school students',
  'Secondary school students',
  'TAFE students',
  'University students',
  'Holds degree or higher, persons',
  'Did not complete year 12, persons',
  'Unemployed, persons',
  'Volunteers, persons',
  'Requires assistance with core activities, persons',
  'Aged 75+ and lives alone, persons',
  'Unpaid carer to person with disability, persons',
  'Unpaid carer of children, persons',
  'Aboriginal or Torres Strait Islander, persons',
  'Born overseas, persons',
  'Born in non-English speaking country, persons',
  'Speaks LOTE at home, persons',
  'Poor English proficiency, persons',
  'Presentations to emergency departments, 2012-13'
]

# Map the normalization base with the list of cols
mapNormBase = {
    "Presentations to emergency departments, 2012-13": presentationNormCols,
    "Area (km^2)": areaNormCols,
    "Number of Dwellings": dwellingNormCols,
    "Number of Households": householdNormCols,
    "Number of families": familyNormCols,
    "Population": populationNormCols
}

# These scores have specific scale (ARIA and IRS), so they won't be in between 0 and 1 (not proportion)
# Included in the norm list to find the weighted average
noPercent = ["ARIA+ (avg)", "ARIA+ (max)", "ARIA+ (min)",
                "IRSD (avg)", "IRSD (max)", "IRSD (min)"]

communities_group_LGA = communities.groupby("LGA").aggregate(aggregate)
for normBase, normCols in mapNormBase.items():
    for col in normCols:
      communities_group_LGA[col] /= communities_group_LGA[normBase]

# Re-derive the "ABS remoteness category", with ordinal encoder
def getABSCat(aria_avg: float):
    if 0 <= aria_avg and aria_avg <= 0.2:
        return 0
    elif 0.2 < aria_avg and aria_avg <= 2.4:
        return 1
    elif 2.4 < aria_avg and aria_avg <= 5.92:
        return 2
    elif 5.92 < aria_avg and aria_avg <= 10.53:
        return 3
    else:
        return 4

communities_group_LGA["ABS remoteness category"] = communities_group_LGA["ARIA+ (avg)"].apply(getABSCat)
columns_keep.append("ABS remoteness category")
    

# Collate all the normalized column in 1 list (to check if they are in 0 and 1)
allNormCols = []
for normBase, normCols in mapNormBase.items():
    for col in normCols:
        if col not in noPercent:
            allNormCols.append(col)

# Check if they are all between 0 and 1
for col in allNormCols:
    mini = communities_group_LGA[col].min()
    maxi = communities_group_LGA[col].max()
    if not ((mini >= 0 and mini <= 1) and (maxi >= 0 and maxi <= 1)):
        print("Wrong scale: ", col)

communities_group_LGA.to_csv('cleanedCommunities.csv')
communities_group_LGA["ABS remoteness category"]

LGA
alpine          2
ararat          1
ballarat        1
banyule         0
basscoast       1
               ..
wodonga         1
wyndham         0
yarra           0
yarraranges     0
yarriambiack    2
Name: ABS remoteness category, Length: 79, dtype: int64

In [37]:
# Preprocess for Housing Price: Grouping of LGA and imputation of data
# Replace '-' with NaN and unneeded/derived attributes
# 
housingPrice = housingPrice.replace('-', 'NaN')
housingPrice = housingPrice.drop(columns=["Change Percentage  13-23"])
for column in housingPrice.columns:
    if column == "LGA" or column == "Locality":
        continue
    housingPrice[column] = housingPrice[column].astype("float")
housingPrice

,Locality,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,LGA
0,ABBOTSFORD,792500.0,862500.0,925000.0,1187500.0,1280000.0,1192500.0,1050000.0,1200000.0,1365000.0,1346000.0,1250000.0,yarra
1,ABERFELDIE,947500.0,1045000.0,1207500.0,1300000.0,1471000.0,1498500.0,1390000.0,1520000.0,1858000.0,1900000.0,1631000.0,mooneevalley
2,AINTREE,NaN,NaN,600000.0,600000.0,571000.0,557500.0,575000.0,616000.0,713500.0,756000.0,748000.0,melton
3,AIREYS INLET,664000.0,625500.0,680000.0,715000.0,737500.0,869000.0,985000.0,1132500.0,1775000.0,1725000.0,1675000.0,surfcoast
4,AIRPORT WEST,532000.0,575000.0,635000.0,742000.0,845000.0,845000.0,795000.0,812500.0,960000.0,911000.0,945000.0,hume
...,...,...,...,...,...,...,...,...,...,...,...,...,...
782,YARRAM,190000.0,202500.0,245000.0,230000.0,225000.0,218000.0,240000.0,260000.0,360000.0,470000.0,395000.0,nillumbik
783,YARRAVILLE,696000.0,710000.0,810000.0,900500.0,1000000.0,1045000.0,971000.0,1030000.0,1185000.0,1210000.0,1100000.0,maribyrnong
784,YARRAWONGA,302500.0,297000.0,300500.0,320000.0,324500.0,345000.0,375000.0,456500.0,564500.0,670000.0,640000.0,moira
785,YEA,260000.0,255000.0,260000.0,305000.0,385000.0,390000.0,435500.0,422500.0,570000.0,617500.0,550000.0,murrindindi


In [38]:
# Get the suburb from communities
suburbs = raw_communities[raw_communities["Community Name"].str.contains("Suburb")].copy()
suburbs["Population"] = suburbs["Population Density"] * suburbs["Area (km^2)"]

# Clean the suburb name and housing locality to match
def clear_region(x: str):
    x = x.lower()
    x = x.replace(" ", "")
    x = x.replace("-", "")
    x = x.split("(")[0]
    return x

housingPrice["Locality"] = housingPrice["Locality"].apply(clear_region)
suburbs["Community Name"] = suburbs["Community Name"].apply(clear_region)

# Get the population of each shared suburb from the communities
def get_population(suburbName: str):
    global suburbs
    if suburbName not in list(suburbs["Community Name"]):
        return None
    else:
        return suburbs[suburbs["Community Name"] == suburbName]["Population"].iloc[0]

housingPrice["Population"] = housingPrice["Locality"].apply(get_population)


# Find the mean population (2023) of the suburbs with the same lga in the housing price dataset
housingPriceMeanPopulation = housingPrice.groupby("LGA")["Population"].mean()

# For those without the community population in the entire LGA, then put it as 1, since we'll take the raw
# average from that
# Impute the missing values with the mean of its group
for idx, row in housingPrice.iterrows():
    if pd.isna(row["Population"]):
        groupMean = housingPriceMeanPopulation.loc[row["LGA"]]
        if pd.isna(groupMean):
            housingPrice.loc[idx, "Population"] = 1
        else:
            housingPrice.loc[idx, "Population"] = groupMean


years = [str(i) for i in range(2013, 2024)]
# Weighted each year by the population
for year in years:
    housingPrice[year] *= housingPrice["Population"]

# Drop community and find the weighted average of housing price
housingPrice = housingPrice.drop(columns=["Locality"])

housingPriceWeightedLGA = housingPrice.groupby("LGA").sum()
for year in years:
    housingPriceWeightedLGA[year] /= housingPriceWeightedLGA["Population"]

housingPriceWeightedLGA = housingPriceWeightedLGA.drop(columns = ["Population"])

# housingPriceWeightedLGA.to_csv("weightedHousingPrice.csv")

In [39]:
# Output to use for eda
# LGA is the index for housingPrices and communities
cleanedHousingPrice = housingPriceWeightedLGA.copy()
cleanedCommunities = communities_group_LGA.copy()
cleanedEgm = egm.copy()
cleanedOffences = offences.copy()

allLGAs = {
    "communities": cleanedCommunities.index.unique(),
    "egm": cleanedEgm["LGA Name"].unique(),
    "housingPrice": cleanedHousingPrice.index.unique(),
    "offences": cleanedOffences["Local Government Area"].unique()
}


commonLGAs = set(allLGAs["communities"]) \
    .intersection(set(allLGAs["egm"]))\
    .intersection(set(allLGAs["housingPrice"]))\
    .intersection(set(allLGAs["offences"]))

allLGA = set(allLGAs["communities"]) \
    .union(set(allLGAs["egm"]))\
    .union(set(allLGAs["housingPrice"]))\
    .union(set(allLGAs["offences"]))

print("Check union of LGA: ", len(commonLGAs))

for lga in allLGA:
    if lga not in commonLGAs:
        if lga in allLGAs["communities"]:
            cleanedCommunities = cleanedCommunities.drop(index=[lga])
        if lga in allLGAs["housingPrice"]:
            cleanedHousingPrice = cleanedHousingPrice.drop(index=[lga])
        if lga in allLGAs["egm"]:
            cleanedEgm = cleanedEgm.drop(index=cleanedEgm[cleanedEgm["LGA Name"] == lga].index)
        if lga in allLGAs["offences"]:
            cleanedOffences = cleanedOffences.drop(index=cleanedOffences[cleanedOffences["Local Government Area"] == lga].index)

# Change the LGA column of all frames to the same name
cleanedEgm = cleanedEgm.rename(columns={"LGA Name": "LGA"})
cleanedOffences = cleanedOffences.rename(columns={"Local Government Area": "LGA"})

# Set lga index for egm
cleanedEgm = cleanedEgm.set_index("LGA")
cleanedEgm = cleanedEgm.drop(columns=["Region"])
columns_keep
allNormCols

Check union of LGA:  56


['Presentations to emergency departments due to injury',
 'Category 4 & 5 emergency department presentations',
 'Commercial (km^2)',
 'Industrial (km^2)',
 'Residential (km^2)',
 'Rural (km^2)',
 'Other (km^2)',
 'Occupied private dwellings',
 'Population in non-private dwellings',
 'Public Housing Dwellings',
 'Dwellings with no motor vehicle',
 'Dwellings with no internet',
 'Equivalent household income <$600/week',
 'Personal income <$400/week, persons',
 'Female-headed lone parent families',
 'Male-headed lone parent families',
 '% residing near PT',
 'Public Hospitals',
 'Private Hospitals',
 'Community Health Centres',
 'Bush Nursing Centres',
 'Allied Health',
 'Alternative Health',
 'Child Protection and Family',
 'Dental',
 'Disability',
 'General Practice',
 'Homelessness',
 'Mental Health',
 'Pharmacies',
 'Aged Care (High Care)',
 'Aged Care (Low Care)',
 'Aged Care (SRS)',
 'Kinder and/or Childcare',
 'Primary Schools',
 'Secondary Schools',
 'P12 Schools',
 'Other Schools

In [42]:
cleanedHousingPrice.to_csv('cleanedHousing.csv')

In [43]:
cleanedHousingPrice

,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
LGA,,,,,,,,,,,
alpine,2.536429e+05,2.458571e+05,2.728571e+05,2.915714e+05,3.235714e+05,3.675714e+05,3.939286e+05,4.434286e+05,5.936429e+05,7.489286e+05,7.375000e+05
ballarat,5.524666e+05,6.099019e+05,6.534769e+05,6.889086e+05,7.288541e+05,7.747367e+05,7.919844e+05,8.178846e+05,9.842962e+05,1.046597e+06,1.031248e+06
banyule,6.533616e+05,7.071675e+05,7.818328e+05,8.743688e+05,9.845694e+05,9.741886e+05,9.475891e+05,1.005577e+06,1.202964e+06,1.192206e+06,1.176132e+06
basscoast,3.203542e+05,3.274167e+05,3.357708e+05,3.531042e+05,3.886250e+05,4.547083e+05,4.651667e+05,5.298750e+05,6.830208e+05,7.736875e+05,7.288125e+05
bawbaw,3.846499e+05,4.035541e+05,4.340382e+05,4.746541e+05,5.032166e+05,5.174603e+05,5.339723e+05,5.916558e+05,7.040088e+05,7.163704e+05,7.789983e+05
bayside,1.060976e+06,1.153000e+06,1.356251e+06,1.544398e+06,1.732025e+06,1.672062e+06,1.572118e+06,1.744736e+06,2.055757e+06,2.040937e+06,1.973979e+06
benalla,2.290000e+05,2.372500e+05,2.237500e+05,2.520000e+05,2.750000e+05,2.745000e+05,3.312500e+05,3.385000e+05,4.470000e+05,4.737500e+05,5.125000e+05
boroondara,1.294164e+06,1.472632e+06,1.789251e+06,1.883535e+06,2.120015e+06,1.983815e+06,1.917591e+06,2.034634e+06,2.362973e+06,2.351284e+06,2.414438e+06
brimbank,4.165666e+05,4.385119e+05,4.851689e+05,5.525590e+05,6.534193e+05,6.719956e+05,6.492047e+05,6.758560e+05,7.620902e+05,7.850776e+05,7.579897e+05
